In [43]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pickle

In [44]:
df = pd.read_csv("Books.csv")
df.head()

C:\Users\wilso\AppData\Local\Temp\ipykernel_13136\2897801158.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Books.csv")


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271358 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [46]:
# Check for missing values in the dataframe
print(df.isnull().sum())

ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64


In [47]:
# Remove null and duplicate values
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)

In [48]:
# Check data types for column 3
print(df["Year-Of-Publication"].apply(type).value_counts())

Year-Of-Publication
<class 'int'>    205820
<class 'str'>     65533
Name: count, dtype: int64


In [49]:
string_years = df[df["Year-Of-Publication"].apply(lambda x: isinstance(x, str))]

# Display only the 'Year-Of-Publication' column for string values
print(string_years["Year-Of-Publication"])

196608    2000
196609    1995
196610    1999
196611    2004
196612    2003
          ... 
262139    1986
262140    1987
262141    1994
262142    1993
262143    1991
Name: Year-Of-Publication, Length: 65533, dtype: object


In [50]:
# Convert 'Year-Of-Publication' from string to int
df['Year-Of-Publication'] = df['Year-Of-Publication'].astype(int)

In [51]:
print(df["Year-Of-Publication"].apply(type).value_counts())

Year-Of-Publication
<class 'int'>    271353
Name: count, dtype: int64
